In [1]:
from sync_asr.riksdag.riksdag_api import RiksdagAPI
from pathlib import Path

In [2]:
speakers = {}
for file in Path("/Users/joregan/Playing/rdapi/api_output").glob("*"):
    rdapi = RiksdagAPI(filename=str(file))
    for se in rdapi.get_speaker_elements():
        if se.speaker_name in speakers:
            speakers[se.speaker_name] += se.duration
        else:
            speakers[se.speaker_name] = se.duration

In [3]:
from sync_asr.riksdag.get_speaker_table import get_people

In [4]:
people = get_people()

In [10]:
from pathlib import Path
wd_path = Path("../data/wikidata/")
tsv_path = wd_path / "person_riksdagid_placeofbirth_pobLabel.tsv"
wikidata = {}
with open(tsv_path) as tsvfile:
    for line in tsvfile.readlines():
        if not line.startswith("http"):
            continue
        parts = line.split("\t")
        rdid = parts[1].strip()
        pob = parts[2].strip()
        pobname = parts[3].strip()
        pers = parts[0].strip()
        wikidata[rdid] = {
            "url": pers,
            "placename": pobname,
            "placeofbirth_url": pob
        }

In [11]:
wikidata["0388849444310"]

{'url': 'http://www.wikidata.org/entity/Q520223',
 'placename': 'Täby',
 'placeofbirth_url': 'http://www.wikidata.org/entity/Q54337'}

In [12]:
OUTBUF = []
OUTBUF.append("Name\tGender\tParty\tTerms\tTotal (sec)\tRiksdag ID\tWikidata\tPlace of birth\tPOB Wikidata")
for person in people.values():
    name = person.get_name()
    gender = person.gender
    party = person.get_party()
    by = person.birth_year
    terms = person.get_merged_terms()
    rd_id = person.id
    if rd_id in wikidata:
        url = wikidata[rd_id]["url"]
        pn = wikidata[rd_id]["placename"]
        pnu = wikidata[rd_id]["placeofbirth_url"]
        extra = f"{url}\t{pn}\t{pnu}"
    else:
        extra = "-\t-\t-"

    if name in speakers:
        OUTBUF.append(f"{name}\t{gender}\t{party}\t{terms}\t{speakers[name] / 1000}\t{rd_id}\t{extra}")


In [13]:
with open("/tmp/persons.tsv", "w") as output:
    for line in OUTBUF:
        output.write(line + "\n")